In [2]:
import pandas as pd
import pickle
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf

In [129]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
import warnings
warnings.filterwarnings('ignore')

ENCODING TECHNIQUE

In [65]:
data = pd.read_csv("ipldata.csv")

In [66]:
data = data.drop(['Unnamed: 0'], axis=1)

In [67]:
data.isnull().sum()

match_id        0
inning          0
batting_team    0
bowling_team    0
over            0
ball            0
batter          0
bowler          0
non_striker     0
batsman_runs    0
extra_runs      0
total_runs      0
is_wicket       0
dtype: int64

In [68]:
numerical_cols = data.select_dtypes(include=['number']).columns.tolist()
numerical_cols

['match_id',
 'inning',
 'over',
 'ball',
 'batsman_runs',
 'extra_runs',
 'total_runs',
 'is_wicket']

In [69]:
categorical_cols = data.select_dtypes(include=['object']).columns.tolist()
categorical_cols

['batting_team', 'bowling_team', 'batter', 'bowler', 'non_striker']

In [70]:
label_encoder = LabelEncoder()

In [71]:
data['batting_team'] = label_encoder.fit_transform(data['batting_team'])

In [72]:
data['bowling_team'] = label_encoder.fit_transform(data['bowling_team'])

In [73]:
data['batter'] = label_encoder.fit_transform(data['batter'])

In [74]:
data['bowler'] = label_encoder.fit_transform(data['bowler'])

In [75]:
data['non_striker'] = label_encoder.fit_transform(data['non_striker'])

In [76]:
with open('batting_team.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

In [77]:
with open('bowling_team.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

In [78]:
with open('batter.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

In [79]:
with open('bowler.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

In [80]:
with open('non_striker.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

In [81]:
data.to_csv("ipl_data.csv")

MODEL TRAINING - SIMPLE REGRESSION

In [82]:
data = pd.read_csv("ipl_data.csv")

In [84]:
data = data.drop(['Unnamed: 0'], axis=1)

In [85]:
data

,match_id,inning,batting_team,bowling_team,over,ball,batter,bowler,non_striker,batsman_runs,extra_runs,total_runs,is_wicket
0,335982,1,8,16,0,1,537,335,103,0,1,1,0
1,335982,1,8,16,0,2,102,335,531,0,0,0,0
2,335982,1,8,16,0,3,102,335,531,0,1,1,0
3,335982,1,8,16,0,4,102,335,531,0,0,0,0
4,335982,1,8,16,0,5,102,335,531,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
260915,1426312,2,8,18,9,5,566,35,631,1,0,1,0
260916,1426312,2,8,18,9,6,639,35,560,1,0,1,0
260917,1426312,2,8,18,10,1,639,462,560,1,0,1,0
260918,1426312,2,8,18,10,2,566,462,631,1,0,1,0


In [114]:
X = data[['batting_team', 'bowling_team', 'batter', 'bowler', 'non_striker', 'batsman_runs', 'extra_runs']]
y = data['total_runs'] 

In [115]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [88]:
scaler = StandardScaler()

In [89]:
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [90]:
model = Sequential()

In [91]:
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))

model.add(Dropout(0.2))

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(16, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(1, activation='linear'))

In [92]:
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [93]:
history = model.fit(X_train, y_train, epochs=10, batch_size=4, validation_data=(X_test, y_test))

Epoch 1/10
52184/52184 ━━━━━━━━━━━━━━━━━━━━ 108s 2ms/step - loss: 0.2610 - mae: 0.2900 - val_loss: 0.0502 - val_mae: 0.0959
Epoch 2/10
52184/52184 ━━━━━━━━━━━━━━━━━━━━ 106s 2ms/step - loss: 0.1080 - mae: 0.2077 - val_loss: 0.0090 - val_mae: 0.0645
Epoch 3/10
52184/52184 ━━━━━━━━━━━━━━━━━━━━ 106s 2ms/step - loss: 0.1022 - mae: 0.1972 - val_loss: 0.0137 - val_mae: 0.0533
Epoch 4/10
52184/52184 ━━━━━━━━━━━━━━━━━━━━ 106s 2ms/step - loss: 0.1005 - mae: 0.1954 - val_loss: 0.0292 - val_mae: 0.0854
Epoch 5/10
52184/52184 ━━━━━━━━━━━━━━━━━━━━ 106s 2ms/step - loss: 0.1015 - mae: 0.1946 - val_loss: 0.0083 - val_mae: 0.0552
Epoch 6/10
52184/52184 ━━━━━━━━━━━━━━━━━━━━ 106s 2ms/step - loss: 0.0989 - mae: 0.1931 - val_loss: 0.0117 - val_mae: 0.0627
Epoch 7/10
52184/52184 ━━━━━━━━━━━━━━━━━━━━ 107s 2ms/step - loss: 0.1005 - mae: 0.1939 - val_loss: 0.0031 - val_mae: 0.0282
Epoch 8/10
52184/52184 ━━━━━━━━━━━━━━━━━━━━ 110s 2ms/step - loss: 0.0992 - mae: 0.1928 - val_loss: 0.0017 - val_mae: 0.0243
Epoch 9/

In [94]:
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")
print(f"Test Mean Absolute Error: {test_mae}")

1631/1631 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.0132 - mae: 0.0710
Test Loss: 0.013599892146885395
Test Mean Absolute Error: 0.07132923603057861


LSTM MODEL

In [95]:
def encode_categorical_columns(X):
    label_encoder = LabelEncoder()
    
    X['batting_team'] = label_encoder.fit_transform(X['batting_team'])
    X['bowling_team'] = label_encoder.fit_transform(X['bowling_team'])
    X['batter'] = label_encoder.fit_transform(X['batter'])
    X['bowler'] = label_encoder.fit_transform(X['bowler'])
    X['non_striker'] = label_encoder.fit_transform(X['non_striker'])

    return X

In [96]:
X_encoded = encode_categorical_columns(X)

In [97]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

In [98]:
X_train = np.reshape(X_train.values, (X_train.shape[0], 1, X_train.shape[1]))  
X_test = np.reshape(X_test.values, (X_test.shape[0], 1, X_test.shape[1])) 

In [99]:
model_lstm = Sequential()

In [100]:
model_lstm.add(LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model_lstm.add(Dropout(0.2))

model_lstm.add(LSTM(32))
model_lstm.add(Dropout(0.2))

model_lstm.add(Dense(1, activation='linear'))

In [101]:
model_lstm.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [102]:
history_lstm = model_lstm.fit(X_train, y_train, epochs=10, batch_size=4, validation_data=(X_test, y_test))

Epoch 1/10
52184/52184 ━━━━━━━━━━━━━━━━━━━━ 138s 3ms/step - loss: 1.6355 - mae: 0.8490 - val_loss: 0.1743 - val_mae: 0.2537
Epoch 2/10
52184/52184 ━━━━━━━━━━━━━━━━━━━━ 142s 3ms/step - loss: 0.3239 - mae: 0.3376 - val_loss: 0.0923 - val_mae: 0.1600
Epoch 3/10
52184/52184 ━━━━━━━━━━━━━━━━━━━━ 136s 3ms/step - loss: 0.2357 - mae: 0.2773 - val_loss: 0.0688 - val_mae: 0.1199
Epoch 4/10
52184/52184 ━━━━━━━━━━━━━━━━━━━━ 135s 3ms/step - loss: 0.2255 - mae: 0.2631 - val_loss: 0.0843 - val_mae: 0.1226
Epoch 5/10
52184/52184 ━━━━━━━━━━━━━━━━━━━━ 135s 3ms/step - loss: 0.1910 - mae: 0.2400 - val_loss: 0.3009 - val_mae: 0.4823
Epoch 6/10
52184/52184 ━━━━━━━━━━━━━━━━━━━━ 135s 3ms/step - loss: 0.2011 - mae: 0.2492 - val_loss: 0.1077 - val_mae: 0.1262
Epoch 7/10
52184/52184 ━━━━━━━━━━━━━━━━━━━━ 136s 3ms/step - loss: 0.2256 - mae: 0.2442 - val_loss: 0.1459 - val_mae: 0.1622
Epoch 8/10
52184/52184 ━━━━━━━━━━━━━━━━━━━━ 136s 3ms/step - loss: 0.2160 - mae: 0.2363 - val_loss: 0.1670 - val_mae: 0.1846
Epoch 9/

In [103]:
test_loss, test_mae = model_lstm.evaluate(X_test, y_test)
print(f"Test Loss (LSTM): {test_loss}")
print(f"Test MAE (LSTM): {test_mae}")

1631/1631 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.1019 - mae: 0.1542
Test Loss (LSTM): 0.10274790972471237
Test MAE (LSTM): 0.15420305728912354


ANN MODEL

In [116]:
model_ann = Sequential()

In [117]:
model_ann.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model_ann.add(Dropout(0.2))

model_ann.add(Dense(32, activation='relu'))
model_ann.add(Dropout(0.2))

model_ann.add(Dense(16, activation='relu'))
model_ann.add(Dropout(0.2))

model_ann.add(Dense(1, activation='linear'))

In [118]:
model_ann.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

In [119]:
history_ann = model_ann.fit(X_train, y_train, epochs=10, batch_size=4, validation_data=(X_test, y_test))

Epoch 1/10
52184/52184 ━━━━━━━━━━━━━━━━━━━━ 107s 2ms/step - loss: 40.0427 - mae: 1.9475 - val_loss: 2.6310 - val_mae: 1.1891
Epoch 2/10
52184/52184 ━━━━━━━━━━━━━━━━━━━━ 106s 2ms/step - loss: 2.6650 - mae: 1.2069 - val_loss: 2.6314 - val_mae: 1.1822
Epoch 3/10
52184/52184 ━━━━━━━━━━━━━━━━━━━━ 106s 2ms/step - loss: 2.6282 - mae: 1.1947 - val_loss: 2.6309 - val_mae: 1.1993
Epoch 4/10
52184/52184 ━━━━━━━━━━━━━━━━━━━━ 142s 2ms/step - loss: 2.6494 - mae: 1.2036 - val_loss: 2.6319 - val_mae: 1.2119
Epoch 5/10
52184/52184 ━━━━━━━━━━━━━━━━━━━━ 106s 2ms/step - loss: 2.6220 - mae: 1.1948 - val_loss: 2.6309 - val_mae: 1.1965
Epoch 6/10
52184/52184 ━━━━━━━━━━━━━━━━━━━━ 106s 2ms/step - loss: 2.6451 - mae: 1.2003 - val_loss: 2.6309 - val_mae: 1.1934
Epoch 7/10
52184/52184 ━━━━━━━━━━━━━━━━━━━━ 106s 2ms/step - loss: 2.6494 - mae: 1.2035 - val_loss: 2.6308 - val_mae: 1.1939
Epoch 8/10
52184/52184 ━━━━━━━━━━━━━━━━━━━━ 107s 2ms/step - loss: 2.6246 - mae: 1.1949 - val_loss: 2.6313 - val_mae: 1.1837
Epoch 9

In [120]:
test_loss, test_mae = model_ann.evaluate(X_test, y_test)
print(f"Test Loss (ANN): {test_loss}")
print(f"Test MAE (ANN): {test_mae}")

1631/1631 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 2.6253 - mae: 1.1877
Test Loss (ANN): 2.6311450004577637
Test MAE (ANN): 1.186254620552063


LSTM model is the best-performing model, and it is likely the most accurate.

In [130]:
model_lstm.save('lstm_model.h5')

print("Model saved")

Model saved


In [131]:
model_lstm = load_model('lstm_model.h5')

print("Model loaded")

Model loaded


In [132]:
X_test_reshaped = np.reshape(X_test.values, (X_test.shape[0], 1, X_test.shape[1]))

In [133]:
model_lstm = load_model('lstm_model.h5')

In [134]:
predictions_lstm = model_lstm.predict(X_test_reshaped)

1631/1631 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step


In [135]:
scaler_y = StandardScaler()
scaler_y.fit(y_train.values.reshape(-1, 1))  
predictions_lstm_rescaled = scaler_y.inverse_transform(predictions_lstm)

In [136]:
test_loss, test_mae = model_lstm.evaluate(X_test_reshaped, y_test)
print(f'Test Loss (LSTM): {test_loss}')
print(f'Test MAE (LSTM): {test_mae}')

1631/1631 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.1019 - mean_absolute_error: 0.1542
Test Loss (LSTM): 0.10274790972471237
Test MAE (LSTM): 0.15420305728912354


In [137]:
print(predictions_lstm_rescaled)

[[3.0499792]
 [2.9355388]
 [7.880351 ]
 ...
 [2.879894 ]
 [2.9733932]
 [1.5265783]]
